<a href="https://colab.research.google.com/github/santhosh12-a/511323106046-AI-Movie/blob/main/511323106046_AI_driven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install scikit-surprise –quiet
 !pip install --upgrade numpy
!pip install scikit-surprise --quiet --no-cache-dir
 !pip uninstall -y numpy
 !pip install numpy
!python -c "import numpy; print(numpy. version )"
 !pip cache purge
 !pip install numpy
 !pip install pandas scikit-learn
from google.colab import files
 Import pandas as pd
import numpy as np
 from sklearn.metrics.pairwise import cosine_similarity
print("Please upload the 'ratings.csv' file from your local folder.")
 uploaded = files.upload()
 ratings_df = pd.read_csv("ratings.csv")
print("\nSample ratings (ratings.csv):")
 print(ratings_df.head())
 print("\nPlease upload the 'movies.csv' file from your local folder.")
 uploaded = files.upload()
 movies_df = pd.read_csv("movies.csv")
print("\nSample movies (movies.csv):")
print(movies_df.head())
user_movie_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')
 user_movie_matrix_filled = user_movie_matrix.fillna(0)
print("User-Movie Matrix Shape:", user_movie_matrix_filled.shape)
user_similarity = cosine_similarity(user_movie_matrix_filled)
 user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
print("User-User Similarity Matrix Sample:")
 print(user_sim_df.head())
 def get_movie_recommendations(user_id, n=5):
similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]
 weighted_ratings = pd.Series(dtype=float)
 for sim_user, similarity in similar_users.items():
 sim_user_ratings = user_movie_matrix.loc[sim_user]
 weighted = sim_user_ratings * similarity
weighted_ratings = weighted_ratings.add(weighted, fill_value=0)
user_rated =
 user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')
 top_movies = weighted_ratings.sort_values(ascending=False).head(n)
 return top_movies, weighted_ratings
 top_recommendations, weighted_ratings =
get_movie_recommendations(user_id=1, n=5)
 print("\nTop 5 Recommendations for User 1:")
 for movie_id, score in top_recommendations.items():
movie_title = movies_df.loc[movies_df['movieId'] == movie_id, 'title'].values
title = movie_title[0] if len(movie_title) > 0 else "Unknown"
 print(f"{title}, Score: {score:.2f}")
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)
 def get_movie_recommendations_filtered(user_id, genre_filter="Action",
min_year=1995, n=5):
 similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]
 weighted_ratings = pd.Series(dtype=float)
 for sim_user, similarity in similar_users.items():
sim_user_ratings = user_movie_matrix.loc[sim_user]
weighted = sim_user_ratings * similarity
weighted_ratings = weighted_ratings.add(weighted, fill_value=0)
 user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
 weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')
 weighted_ratings /= similar_users.sum()
 movie_info = movies_df.copy()
 movie_info['year'] = movie_info['title'].str.extract(r'\((\d{4})\)').astype(float)
 filtered = movie_info
[ (movie_info['movieId'].isin(weighted_ratings.index)) &
 (movie_info['genres'].str.contains(genre_filter, case=False, na=False)) &
(movie_info['year'] >= min_year) ]
 filtered['score'] = filtered['movieId'].map(weighted_ratings)
 top_recs = filtered.sort_values(by='score', ascending=False).head(n)
 return top_recs[['title', 'genres', 'year', 'score']]
 genre_input = "Action"
min_year_input = 1995
recommendations = get_movie_recommendations_filtered(user_id=1, genre_filter=genre_input, min_year=min_year_input)
print(f"\nTop 5 Recommendations for User 1 — Genre: {genre_input}, After {min_year_input}:\n")
 print(recommendations.to_string(index=False))
def get_custom_recommendations(user_id, genre_filters=None, start_year=None, end_year=None, keyword=None, top_n=5):
similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:]
 weighted_ratings = pd.Series(dtype=float)
 for sim_user, similarityin similar_users.items():
 sim_user_ratings = user_movie_matrix.loc[sim_user]
weighted = sim_user_ratings * similarity
weighted_ratings = weighted_ratings.add(weighted, fill_value=0)
 user_rated = user_movie_matrix.loc[user_id][user_movie_matrix.loc[user_id].notnull()].index
weighted_ratings = weighted_ratings.drop(user_rated, errors='ignore')
 weighted_ratings /= similar_users.sum()
 movie_info = movies_df.copy()
movie_info['year'] = movie_info['title'].str.extract(r'\((\d{4})\)').astype(float)
if genre_filters:
 genre_filters = [g.strip().lower() for g in genre_filters]
 genre_mask = movie_info['genres'].apply(lambda g: any(gen in g.lower() for gen in genre_filters))
 movie_info = movie_info[genre_mask]
if start_year:
 movie_info = movie_info[movie_info['year'] >= start_year]
 if end_year:
movie_info = movie_info[movie_info['year'] <= end_year]
if keyword:
 movie_info = movie_info[movie_info['title'].str.contains(keyword, case=False, na=False)]
movie_info['score'] = movie_info['movieId'].map(weighted_ratings)
 movie_info = movie_info.dropna(subset=['score'])
top_recs = movie_info.sort_values('score', ascending=False).head(top_n)
 return top_recs[['title', 'genres', 'year', 'score']]
 user_id = 1
 genre_input = input("Enter preferred genres (comma-separated, e.g., Action,Comedy): ").lower()
 start_year = int(input("Enter start year (e.g. 1995): "))
 end_year = int(input("Enter end year (e.g. 2020): "))
 use_keyword = input("Do you want to filter by a keyword in the title? (yes/no): ").strip().lower()
 keyword = None
 if use_keyword == 'yes':
 keyword = input("Enter keyword in title (e.g., war, love): ")
 movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)
available_genres = set()
for genre_list in movies_df['genres'].str.split('|'):
 available_genres.update(genre_list)
 available_years = set(movies_df['year']
) genre_list = genre_input.split(',')
 invalid_genres = [genre for genre in genre_list if genre not in available_genres]
 if invalid_genres:
 print(f"\nThe following genres are not found in the dataset: {', '.join(invalid_genres)}")
 if start_year not in available_years or end_year not in available_years:
 print(f"\nOne or both of the years are not valid. Valid years are between {int(min(available_years))} and {int(max(available_years))}.")
 if not invalid_genres and start_year in available_years and end_year in available_years:
recommendations = get_custom_recommendations(
 user_id=user_id,
genre_filters=genre_list,
 start_year=start_year, end_year=end_year,
 keyword=keyword,
 top_n=5
 )
print("\nCustom Top 5 Recommendations:\n")
print(recommendations.to_string(index=False))
 else:
 print("\nPlease fix the invalid input and try again.")